In [1]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K 
from keras.optimizers import SGD
from keras.callbacks import CSVLogger
import numpy as np

#loading inceptionv3
base_model = InceptionV3(weights='imagenet', include_top=False)

In [2]:
#creating model
x = base_model.output

#adding top layers
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation = 'relu')(x)
pred = Dense(3,activation = 'softmax')(x)

#creating logger
csv_log = CSVLogger('log.csv', separator=',', append=True)

model = Model(base_model.input, pred)

In [3]:
#freezing layers of base_model: only fully connected layers are trained
for layer in base_model.layers:
    layer.trainable = False

In [4]:
#creating datagenerators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('train_00/train', target_size=(299,299), \
                                                    batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory('train_00/valid', target_size=(299,299), \
                                                    batch_size=32, class_mode='categorical')
test_generator = valid_datagen.flow_from_directory('train_00/test', target_size=(299,299), \
                                                    batch_size=32, class_mode='categorical')

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit_generator(train_generator, validation_data=valid_generator, epochs=10, steps_per_epoch=int(1200/32)+1, validation_steps=int(300/32)+1, callbacks = [csv_log])

Epoch 1/10
38/38 [==============================] - 24s 626ms/step - loss: 1.2547 - accuracy: 0.5142 - val_loss: 0.5071 - val_accuracy: 0.6733
Epoch 2/10
38/38 [==============================] - 15s 396ms/step - loss: 0.6727 - accuracy: 0.7325 - val_loss: 1.1199 - val_accuracy: 0.6400
Epoch 3/10
38/38 [==============================] - 17s 460ms/step - loss: 0.5900 - accuracy: 0.7558 - val_loss: 0.5884 - val_accuracy: 0.7033
Epoch 4/10
38/38 [==============================] - 17s 457ms/step - loss: 0.5457 - accuracy: 0.7817 - val_loss: 1.3260 - val_accuracy: 0.7233
Epoch 5/10
38/38 [==============================] - 17s 459ms/step - loss: 0.4946 - accuracy: 0.7908 - val_loss: 0.5097 - val_accuracy: 0.6700
Epoch 6/10
38/38 [==============================] - 18s 463ms/step - loss: 0.5030 - accuracy: 0.7892 - val_loss: 0.8937 - val_accuracy: 0.6633
Epoch 7/10
38/38 [==============================] - 18s 466ms/step - loss: 0.5037 - accuracy: 0.8025 - val_loss: 0.5106 - val_accuracy: 0.7233

In [7]:
#unfreezing top convolutional layers
for layer in model.layers[172:]:
       layer.trainable = True

In [8]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'], loss='categorical_crossentropy')

In [9]:
model.fit_generator(train_generator,steps_per_epoch=int(1200/32)+1,validation_data=valid_generator,validation_steps=int(300/32)+1,epochs=3, callbacks = [csv_log])

Epoch 1/3
38/38 [==============================] - 31s 823ms/step - loss: 0.4151 - accuracy: 0.8317 - val_loss: 0.6762 - val_accuracy: 0.6667
Epoch 2/3
38/38 [==============================] - 18s 484ms/step - loss: 0.3411 - accuracy: 0.8633 - val_loss: 0.9369 - val_accuracy: 0.7033
Epoch 3/3
38/38 [==============================] - 18s 484ms/step - loss: 0.3308 - accuracy: 0.8608 - val_loss: 0.8672 - val_accuracy: 0.7033


In [10]:
model.evaluate_generator(test_generator, steps=int(300/32)+1)

[0.8467339873313904, 0.6933333277702332]